### 4.1 누락된 데이터 다루기

#### 4.1.1 테이블 형태 데이터에서 누락된 값 식별

In [1]:
import pandas as pd
from io import StringIO

csv_data = '''
A,B,C,D
1.0, 2.0, 3.0, 4.0
5.0, 6.0,, 8.0
10.0, 11.0, 12.0'''

df = pd.read_csv(StringIO(csv_data))
df

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [2]:
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

#### 4.1.2 누락된 값이 있는 훈련 샘플이나 특성 제외

In [3]:
df.dropna(axis=0) # NaN이 있는 행 제거

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [4]:
df.dropna(axis=1) # NaN이 있는 열 제거

,A,B
0,1.0,2.0
1,5.0,6.0
2,10.0,11.0


In [5]:
df.dropna(how='all') # 모든 열이 NaN일 떄만 행을 삭제

,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,NaN,8.0
2,10.0,11.0,12.0,NaN


In [6]:
df.dropna(thresh=4) # NaN이 아닌 값이 4개보다 작은 행을 삭제

,A,B,C,D
0,1.0,2.0,3.0,4.0


In [7]:
df.dropna(subset=['C']) # 특저 ㅇ열에 NaN이 있는 행만 삭제

,A,B,C,D
0,1.0,2.0,3.0,4.0
2,10.0,11.0,12.0,NaN


#### 4.1.3 누락된 값 대체

In [8]:
# 방법1
from sklearn.impute import SimpleImputer
import numpy as np

imr = SimpleImputer(missing_values=np.nan, strategy='mean') # 각 특성 열에서 계산한 평균으로 바꿈, strategy에는 median, most_frequent이 있음
imr = imr.fit(df.values)
imputed_data = imr.transform(df.values)
imputed_data

array([[ 1. ,  2. ,  3. ,  4. ],
       [ 5. ,  6. ,  7.5,  8. ],
       [10. , 11. , 12. ,  6. ]])

In [9]:
# 방법2
df.fillna(df.mean())

df.fillna(method='bfill') # 누락된 값을 다음 행의 값으로 채움
df.fillna(method='ffill') # 누락된 값을 이전 행의 값으로 채움
df.fillna(method='ffill', axis=1) # 행 대신 열의 값으로 함

C:\Users\cho03\AppData\Local\Temp\ipykernel_12048\529794127.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='bfill') # 누락된 값을 다음 행의 값으로 채움
C:\Users\cho03\AppData\Local\Temp\ipykernel_12048\529794127.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill') # 누락된 값을 이전 행의 값으로 채움
C:\Users\cho03\AppData\Local\Temp\ipykernel_12048\529794127.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', axis=1) # 행 대신 열의 값으로 함


,A,B,C,D
0,1.0,2.0,3.0,4.0
1,5.0,6.0,6.0,8.0
2,10.0,11.0,12.0,12.0


#### 4.1.4 사이킷런 추정기 API 익히기

변환기 API : 데이터 변환에 관련된 파이썬 클래스 구현 가능   
- fit : 훈련 데이터에서 모델 파라미터 학습
- transform : 학습한 파라미터로 데이터를 변환
- 변환하려는 데이터 배열은 모델 학습에 사용한 데이터의 특성 개수와 같아야 함

### 4.2 범주형 데이터 다루기

- 순서가 있는 범주형 데이터
- 순서가 없는 범주형 데이터

#### 4.2.1 판다스를 사용한 범주형 데이터 인코딩

In [10]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color','size','price','classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class2
1,red,L,13.5,class1
2,blue,XL,15.3,class2


#### 4.2.2 순서가 있는 특성 매핑

In [11]:
size_mapping = {'XL':3, 'L':2,'M':1}
df['size'] = df['size'].map(size_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [12]:
# 원상복구
inv_size_mapping = {v: k for k, v, in size_mapping.items()}
df['size'].map(inv_size_mapping)

0     M
1     L
2    XL
Name: size, dtype: object

#### 4.2.3 클래스 레이블 인코딩(순서가 없는 범주형 데이터)

In [13]:
## 방법1
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping

{'class1': 0, 'class2': 1}

In [14]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,1
1,red,2,13.5,0
2,blue,3,15.3,1


In [15]:
# 원상복구
inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

,color,size,price,classlabel
0,green,1,10.1,class2
1,red,2,13.5,class1
2,blue,3,15.3,class2


In [16]:
## 다른 방법
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([1, 0, 1])

In [17]:
class_le.inverse_transform(y)

array(['class2', 'class1', 'class2'], dtype=object)

#### 4.2.4 순서가 없는 특성에 원-핫 인코딩 적용

In [18]:
X = df[['color', 'size', 'price']].values
X
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:,0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

**순서가 없는 데이터를 0, 1, 2 이런 식으로 바꾸면 순서가 있다고 판단 -> 원-핫 인코딩 사용**

In [19]:
# 방법1
from sklearn.preprocessing import OneHotEncoder
X = df[['color', 'size', 'price']].values
color_ohe = OneHotEncoder()
color_ohe.fit_transform(X[:,0].reshape(-1, 1)).toarray() # 배열의 다른 두 열을 수정하지 않기 위해 하나의 열(X[:,0].reshape(-1, 1))에만 적용

array([[0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.]])

In [20]:
from sklearn.compose import ColumnTransformer # 여러 개의 특성이 있는 배열에서 특정 열만 변환하기 위해 사용

X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([
    ('onehot', OneHotEncoder(dtype=int), [0]), # 첫번째 열만 변환
    ('nothing','passthrough',[1,2]) # 나머지 두 열은 변경하지 않고 그대로 두기 위해 'passthrough'로 지정
])
c_transf.fit_transform(X)

array([[0, 1, 0, 1, 10.1],
       [0, 0, 1, 2, 13.5],
       [1, 0, 0, 3, 15.3]], dtype=object)

In [21]:
# 방법2(판다스 get_dummies)
pd.get_dummies(df[['price', 'color', 'size']])

# pd.get_dummies(df[['price', 'color', 'size']], columns=['size]) # columns 매개변수를 사용하여 변환하려는 특성을 구체적으로 지정


,price,size,color_blue,color_green,color_red
0,10.1,1,False,True,False
1,13.5,2,False,False,True
2,15.3,3,True,False,False


**※ 원-핫 인코딩할 때 *다중 공선성*  문제를 유념 ※**  
  
특성 간의 상관관계가 높으면 역행렬을 계산하기 어려워 수치적으로 불안정   
-> 상관관계를 감소하려면 원-핫 인코디오딘 배열에서 특성 열 하나를 삭제(문제는 없음)

In [22]:
pd.get_dummies(df[['price', 'color', 'size']], drop_first=True)

,price,size,color_green,color_red
0,10.1,1,True,False
1,13.5,2,False,True
2,15.3,3,False,False


In [23]:
from sklearn.compose import ColumnTransformer # 여러 개의 특성이 있는 배열에서 특정 열만 변환하기 위해 사용

color_ohe = OneHotEncoder(categories='auto', drop='first')
X = df[['color', 'size', 'price']].values
c_transf = ColumnTransformer([
    ('onehot', color_ohe, [0]), # 첫번째 열만 변환
    ('nothing','passthrough',[1,2]) # 나머지 두 열은 변경하지 않고 그대로 두기 위해 'passthrough'로 지정
])
c_transf.fit_transform(X)

array([[1.0, 0.0, 1, 10.1],
       [0.0, 1.0, 2, 13.5],
       [0.0, 0.0, 3, 15.3]], dtype=object)

**순서가 있는 특성 인코딩하기**  
   
ex) M, L, XL -> 'x > M'과 'x > L'로 나눌 수 있음

In [24]:
df = pd.DataFrame([
    ['green', 'M', 10.1, 'class2'],
    ['red', 'L', 13.5, 'class1'],
    ['blue', 'XL', 15.3, 'class2']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

df['x > M'] = df['size'].apply(lambda x : 1 if x in {'L', 'XL'} else 0)
df['x > L'] = df['size'].apply(lambda x : 1 if x == 'XL' else 0)
del df['size']
df

,color,price,classlabel,x > M,x > L
0,green,10.1,class2,0,0
1,red,13.5,class1,1,0
2,blue,15.3,class2,1,1


### 4.3 데이터셋을 훈련 데이터셋과 테스트 데이터셋으로 나누기

In [26]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', '0D280/0D315 of diluted wines', 'Proline']

print('클래스 레이블', np.unique(df_wine['Class label']))

클래스 레이블 [1 2 3]


In [27]:
from sklearn.model_selection import train_test_split

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

### 4.4 특성 스케일 맞추기

### 4.5 유용한 특성 선택

### 4.6 랜덤 포레스트의 특성 중요도 사용